In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import copy

# scikit learn
from sklearn.model_selection import train_test_split

# tensorflow-keras
import tensorflow as tf
from tensorflow import keras

In [ ]:
# load the dataset and get labelled subset
fashion_mnist = keras.datasets.fashion_mnist.load_data()
(X_train, y_train), (X_test, y_test) = fashion_mnist
X_train = X_train.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255

In [ ]:
# checking no. of class instance in labelled set
for k, v in sorted(Counter(y_train_labelled).items()):
    print(f"Class {k} has {v} instances")

## 1. Sklearn
#### a. Label Propagation

In [ ]:
def create_unlabelled_targets(data, target, package):
    if package == "sklearn":
        indices = np.random.permutation(target.size)[:20_000]
        y = copy.deepcopy(target).astype(np.int8)
        X, y = data[indices], y[indices]
        y[:-500] = -1        
        return X.reshape(-1, 784), y
    
    elif package == 'keras':
        X_train_labelled = data[-500:]
        y_train_labelled = target[-500:]
        return X_train_labelled, y_train_labelled

In [ ]:
sk_train, sk_target = create_unlabelled_targets(X_train, y_train, package="sklearn")

In [ ]:
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from scipy.sparse import csr_matrix

sk_train_sparse = csr_matrix(sk_train)

In [ ]:
label_prop_model = LabelPropagation()
label_spread_model = LabelSpreading()

In [ ]:
label_prop_model.fit(sk_train, sk_target)

In [ ]:
label_spread_model.fit(sk_train, sk_target)

In [ ]:
f = label_prop_model.predict(X_test.reshape(-1, 784))
g = label_spread_model.predict(X_test.reshape(-1, 784))

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, f)

In [ ]:
accuracy_score(y_test, g)

In [ ]:
from ske